<a href="https://colab.research.google.com/github/abufirziazka-tech/Dashboard_Analisis_Sentimen/blob/main/DASHBOARD_ANALISIS_SENTIMEN_WACANA_TOLERANSI_BERAGAMA_v10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================
# ✅ FINAL SINGLE-CELL (COLAB)
# 📘 DASHBOARD ANALISIS SENTIMEN WACANA TOLERANSI BERAGAMA — FULL PIPELINE
# Versi: v10 (Ayat Penguat tabel + Peringatan Kontekstual historis)
# ===============================

# 0) IMPORT PERTAMA
import re
import sys, subprocess, importlib, os
from io import BytesIO
from PIL import Image
import matplotlib
matplotlib.use("Agg")  # Non-GUI backend untuk Gradio

def pip_install(pkgs):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + pkgs)

# Daftar dependensi
reqs = [
    ("pyarabic", "pyarabic"),
    ("gradio", "gradio>=4.36.0"),
    ("transformers", "transformers>=4.41.0"),
    ("pandas", "pandas"),
    ("numpy", "numpy"),
    ("matplotlib", "matplotlib"),
    ("PIL", "Pillow"),
    ("torch", "torch"),
    ("scikit_learn", "scikit-learn"),
    ("gensim", "gensim"),
    ("sentence_transformers", "sentence-transformers"),
    ("seaborn", "seaborn")
]
for mod, pkg in reqs:
    try:
        if mod == "PIL":
            importlib.import_module("PIL")
        elif mod == "scikit_learn":
            importlib.import_module("sklearn")
        else:
            importlib.import_module(mod.replace("_", ""))
    except ImportError:
        pip_install([pkg])

# 1) IMPORT UTAMA
import gradio as gr
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity
from gensim import corpora
from gensim.models import LdaModel
from sentence_transformers import SentenceTransformer
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import matplotlib.pyplot as plt
import seaborn as sns

# Cek GPU
device = 0 if torch.cuda.is_available() else -1

# 2) MODEL CONFIGURATION
MODEL_ARABERT = "CAMeL-Lab/bert-base-arabic-camelbert-msa-sentiment"
MODEL_CAMEL = "CAMeL-Lab/bert-base-arabic-camelbert-msa-sentiment"
MODEL_SBERT = "paraphrase-multilingual-MiniLM-L12-v2"

# Status loading (default False / None)
arabert_loaded = camel_loaded = sbert_loaded = False
arabert_err = camel_err = sbert_err = None

# 3) DATA KONTEKS & FRASE SENSITIF (lengkap — dari input user)
SENSITIVE_VERSES = {
    # 1) لا إكراه في الدين
    "لا إكراه في الدين": {
        "warning": "🟢 Ayat toleransi utama: tidak ada paksaan dalam agama.",
        "context": "QS. Al-Baqarah:256 harus dibaca bersama QS. Yunus:99; menunjukkan bahwa iman adalah pilihan bebas.",
        "maqashid": ["Hifz ad-Din", "Hifz al-Aql"],
        "override_sentiment": {"positive": 0.95, "neutral": 0.05, "negative": 0.0},
        "intertextual": {
            "related_verses": ["QS. Yunus:99", "QS. Al-Kahf:29", "QS. Al-Ghashiyah:21-22"],
            "explanation": "Ayat ini merupakan dasar teologis bagi kebebasan beragama dalam Islam, menegaskan bahwa keyakinan tidak bisa dipaksakan."
        },
        "historical_context": "Turun sebagai respons terhadap permintaan seorang sahabat untuk memaksa anak-anaknya yang kafir masuk Islam."
    },
    # 2) لكم دينكم ولي دين
    "لكم دينكم ولي دين": {
        "warning": "🟢 Ayat toleransi fundamental: penghormatan terhadap perbedaan keyakinan.",
        "context": "Diturunkan sebagai penegasan damai terhadap kaum musyrik.",
        "maqashid": ["Hifz ad-Din"],
        "override_sentiment": {"positive": 0.95, "neutral": 0.05, "negative": 0.0},
        "intertextual": {
            "related_verses": ["QS. Al-Baqarah:256", "QS. Al-An'am:108"],
            "explanation": "Ayat ini menunjukkan pengakuan terhadap keberadaan agama lain tanpa kompromi dalam keyakinan tauhid."
        },
        "historical_context": "Turun sebagai respons terhadap tawaran sinkretisme dari kaum musyrik Quraisy."
    },
    # 3) يا أيها الناس إنا خلقناكم
    "يا أيها الناس إنا خلقناكم": {
        "warning": "🟢 Prinsip kesetaraan manusia berdasarkan takwa, bukan ras atau suku.",
        "context": "QS. Al-Hujurat:13 menekankan bahwa nilai manusia ditentukan oleh ketakwaan, bukan latar belakang.",
        "maqashid": ["Hifz an-Nafs", "Hifz al-'Ird"],
        "override_sentiment": {"positive": 0.93, "neutral": 0.07, "negative": 0.0},
        "intertextual": {
            "related_verses": ["QS. Al-Furqan:41", "QS. Az-Zumar:67"],
            "explanation": "Menjadi landasan etika sosial Islam: tidak ada diskriminasi berdasarkan etnisitas."
        }
    },
    # 4) لا ينهاكم الله عن الذين لم يقاتلوكم
    "لا ينهاكم الله عن الذين لم يقاتلوكم": {
        "warning": "🟢 Izin berbuat baik kepada non-Muslim yang damai.",
        "context": "QS. Al-Mumtahanah:8 memberi ruang bagi hubungan sosial positif dengan non-muslim yang tidak memusuhi Islam.",
        "maqashid": ["Hifz an-Nafs", "Hifz al-Mal"],
        "override_sentiment": {"positive": 0.94, "neutral": 0.06, "negative": 0.0},
        "intertextual": {
            "related_verses": ["QS. Al-Balad:17", "QS. Al-Insan:8"],
            "explanation": "Menunjukkan bahwa kebaikan universal (birr) tidak dibatasi oleh agama lawan bicara."
        },
        "historical_context": "Turun saat sebagian Muslim ingin mencela dewa-dewa Arab kafir, sehingga dicegah oleh ayat ini."
    },
    # 5) ولا تجادلوا أهل الكتاب إلا بالتي هي أحسن
    "ولا تجادلوا أهل الكتاب إلا بالتي هي أحسن": {
        "warning": "🟢 Etika dialog antarumat: gunakan cara terbaik dalam berdiskusi.",
        "context": "QS. Al-Ankabut:46 menekankan pentingnya akhlak mulia dalam dakwah kepada Ahli Kitab.",
        "maqashid": ["Hifz al-'Aql", "Hifz ad-Din"],
        "override_sentiment": {"positive": 0.92, "neutral": 0.08, "negative": 0.0},
        "intertextual": {
            "related_verses": ["QS. An-Nahl:125", "QS. Luqman:17"],
            "explanation": "Dialog harus didasarkan pada hikmah dan nasihat yang baik, bukan provokasi."
        }
    },
    # 6) ولو شاء ربك لآمن من في الأرض كلهم جميعاً
    "ولو شاء ربك لآمن من في الأرض كلهم جميعاً": {
        "warning": "🟢 Penegasan bahwa iman adalah urusan Allah, bukan hasil paksaan.",
        "context": "QS. Yunus:41 menjadi argumen kuat melawan doktrin pemaksaan agama.",
        "maqashid": ["Hifz ad-Din", "Hifz al-Iradah"],
        "override_sentiment": {"positive": 0.94, "neutral": 0.06, "negative": 0.0},
        "intertextual": {
            "related_verses": ["QS. Al-Baqarah:256", "QS. Al-Ghashiyah:22"],
            "explanation": "Allah memiliki kemampuan mengimanikan seluruh manusia, tetapi memilih memberi kebebasan — bukti adanya free will."
        }
    },
    # 7) ولا تسبوا الذين يدعون من دون الله
    "ولا تسبوا الذين يدعون من دون الله": {
        "warning": "🟢 Larangan provokasi terhadap pemeluk agama lain.",
        "context": "QS. Al-An'am:108 melarang ejekan terhadap simbol agama lain karena bisa memicu balas dendam terhadap Allah.",
        "maqashid": ["Hifz an-Nafs", "Hifz al-Mal"],
        "override_sentiment": {"positive": 0.91, "neutral": 0.09, "negative": 0.0},
        "intertextual": {
            "related_verses": ["QS. Al-Kafirun:6", "QS. Al-Mumtahanah:8"],
            "explanation": "Etika dialog dalam Islam mengedepankan pencegahan konflik, bukan eskalasi."
        },
        "historical_context": "Turun saat sebagian Muslim ingin mencela dewa-dewa Arab kafir, sehingga dicegah oleh ayat ini."
    },
    # 8) ما أنت عليهم بمضطر
    "ما أنت عليهم بمضطر": {
        "warning": "🟢 Penegasan bahwa dakwah bukan paksaan.",
        "context": "QS. Al-Ghashiyah:22 menegaskan bahwa Rasul hanya bertugas menyampaikan, bukan memaksa.",
        "maqashid": ["Hifz ad-Din", "Hifz al-Aql"],
        "override_sentiment": {"positive": 0.94, "neutral": 0.06, "negative": 0.0},
        "intertextual": {
            "related_verses": ["QS. Al-Baqarah:256", "QS. Yunus:41"],
            "explanation": "Peran nabi adalah tabligh, bukan ikrah — konsisten dalam seluruh Al-Qur'an."
        }
    },
    # 9) ولقد كرّمنا بني آدم
    "ولقد كرّمنا بني آدم": {
        "warning": "🟢 Penghargaan universal terhadap martabat manusia.",
        "context": "QS. Al-Isra:70 menegaskan bahwa semua anak Adam dimuliakan, tanpa pandang agama.",
        "maqashid": ["Hifz an-Nafs", "Hifz al-'Ird"],
        "override_sentiment": {"positive": 0.95, "neutral": 0.05, "negative": 0.0},
        "intertextual": {
            "related_verses": ["QS. Al-Hujurat:13", "QS. Al-Anbiya:107"],
            "explanation": "Martabat manusia bersifat inheren, bukan kondisional berdasarkan keyakinan."
        }
    },
    # 10) وألف بين قلوبهم
    "وألف بين قلوبهم": {
        "warning": "🟢 Doa Nabi untuk persatuan umat.",
        "context": "QS. Al-Hashr:10 menunjukkan harapan persaudaraan antarumat.",
        "maqashid": ["Hifz an-Nafs", "Hifz ad-Din"],
        "override_sentiment": {"positive": 0.91, "neutral": 0.09, "negative": 0.0}
    }
}

INTERTEXTUAL_DB = {
    "قتل": {
        "related_verses": ["QS. Al-Maidah:32", "QS. Al-Isra:33", "QS. Al-Furqan:68"],
        "theme": "Sanctity of Life",
        "explanation": "Ayat-ayat tentang pembunuhan saling melengkapi dalam membentuk etika kehidupan dalam Islam."
    },
    "السلام": {
        "related_verses": ["QS. Al-Anfal:61", "QS. Al-Hujurat:10-13", "QS. An-Nisa:94"],
        "theme": "Peace and Reconciliation",
        "explanation": "Konsep perdamaian dalam Islam bersifat komprehensif dan multidimensi."
    },
    "العدل": {
        "related_verses": ["QS. Al-Maidah:8", "QS. An-Nahl:90", "QS. Al-Hujurat:13"],
        "theme": "Justice and Equality",
        "explanation": "Keadilan adalah nilai sentral dalam Islam, berlaku untuk semua manusia tanpa diskriminasi."
    },
    "البر": {
        "related_verses": ["QS. Al-Mumtahanah:8", "QS. Al-Balad:17", "QS. Al-Insan:8"],
        "theme": "Kindness to Others",
        "explanation": "Berbuat baik kepada non-Muslim yang damai adalah ajaran inti dalam Islam."
    },
    "الرحمة": {
        "related_verses": ["QS. Al-Anbiya:107", "QS. Al-A'raf:156", "QS. Luqman:13"],
        "theme": "Divine and Human Mercy",
        "explanation": "Rahmat Allah universal, mencakup semua makhluk, bukan hanya Muslim."
    },
    "التَّعَارُف": {
        "related_verses": ["QS. Al-Hujurat:13", "QS. Al-Rum:22", "QS. Fussilat:53"],
        "theme": "Mutual Recognition",
        "explanation": "Perbedaan suku dan bangsa adalah sunnatullah untuk saling mengenal."
    },
    "الْمُوَاطَنَة": {
        "related_verses": ["QS. Al-Mumtahanah:8", "QS. Al-Kafirun:6", "QS. Al-Hujurat:13"],
        "theme": "Citizenship and Coexistence",
        "explanation": "Islam mengakui kehidupan bersama sebagai warga negara yang setara."
    },
    "الْحُرِّيَّة": {
        "related_verses": ["QS. Al-Baqarah:256", "QS. Yunus:41", "QS. Al-Ghashiyah:22"],
        "theme": "Freedom of Belief",
        "explanation": "Kebebasan beragama adalah hak asasi yang dilindungi syariah."
    },
    "الْحِكْمَة": {
        "related_verses": ["QS. An-Nahl:125", "QS. Luqman:17", "QS. Al-Imran:159"],
        "theme": "Wisdom in Da'wah",
        "explanation": "Dakwah harus disampaikan dengan hikmah dan nasihat yang baik."
    },
    "الْمُجَادَلَة بِالَّتِي هِيَ أَحْسَنُ": {
        "related_verses": ["QS. Al-Ankabut:46", "QS. Al-Zumar:18", "QS. Ghafir:33"],
        "theme": "Civil Dialogue",
        "explanation": "Dialog dengan Ahli Kitab harus dilakukan dengan cara terbaik."
    }
}

HISTORICAL_CONTEXT_DB = {
    "ميثاق": {
        "context": "Konsep perjanjian dalam Islam didasarkan pada Piagam Madinah yang mengakui pluralitas agama.",
        "significance": "Landasan historis untuk masyarakat multireligius dalam sejarah Islam."
    },
    "أهل الكتاب": {
        "context": "Terminologi ini berkembang dalam dialog antara Muslim dengan Yahudi dan Kristen di Madinah.",
        "significance": "Menunjukkan pengakuan khusus terhadap komunitas agama yang memiliki kitab suci."
    },
    "ذمة": {
        "context": "Status hukum non-Muslim yang hidup di bawah perlindungan negara Islam.",
        "significance": "Menjamin hak-hak sipil dan keamanan jiwa serta harta."
    },
    "جزية": {
        "context": "Pajak yang dibayar oleh non-Muslim sebagai ganti layanan negara dan pembebasan dari wajib militer.",
        "significance": "Bagian dari kontrak sosial dalam masyarakat Islam."
    },
    "دار السلام": {
        "context": "Istilah untuk wilayah damai di mana hukum Islam diterapkan secara adil bagi semua warga.",
        "significance": "Konsep inklusivitas dan keamanan kolektif."
    }
}

PHILOSOPHICAL_CONCEPTS = {
    "التسامح": {
        "concept": "Toleransi dalam Islam bukan sekadar membiarkan, tetapi aktif menghargai perbedaan, menjaga martabat, dan membangun kedamaian antarumat beragama.",
        "theological_basis": "Berdasarkan pada konsep 'لا إكراه في الدين' (QS. Al-Baqarah 2:256) dan kemuliaan manusia sebagai makhluk bernilai (karamah insaniyah), sebagaimana ditegaskan dalam QS. Al-Isra 17:70."
    },
    "العدل": {
        "concept": "Keadilan dalam Islam bersifat universal, mencakup semua manusia tanpa diskriminasi, baik Muslim maupun non-Muslim, dalam hak, perlakuan, dan hukum.",
        "theological_basis": "Merupakan salah satu sifat Allah (Al-Adl) dan prinsip fundamental syariah. Ditegaskan dalam QS. Al-Maidah 5:8: 'Dan janganlah kebencianmu terhadap suatu kaum mendorongmu untuk berlaku tidak adil.'"
    },
    "الرحمة": {
        "concept": "Rahmat dalam Islam mencakup semua makhluk, tidak terbatas pada Muslim saja, melainkan rahmat yang universal bagi alam semesta.",
        "theological_basis": "Terdapat dalam konsep 'رَحْمَةً لِّلْعَالَمِينَ' (rahmatan lil-'alamin) yang menjadi tujuan utama diutusnya Nabi Muhammad (QS. Al-Anbiya 21:107)."
    },
    "التَّعَارُف": {
        "concept": "Manusia diciptakan berbangsa-bangsa dan bersuku-suku bukan untuk saling memusuhi, melainkan agar saling mengenal, memahami, dan mempererat hubungan kemanusiaan.",
        "theological_basis": "Ditegaskan dalam QS. Al-Hujurat 49:13: '...وَجَعَلْنَاكُمْ شُعُوبًا وَقَبَائِلَ لِتَعَارَفُوا'. Perbedaan adalah sunnatullah yang harus dihargai."
    },
    "السَّلَام": {
        "concept": "Islam sebagai agama perdamaian menekankan pentingnya hidup damai, dialog, dan penolakan terhadap permusuhan yang tidak perlu, terutama dengan pemeluk agama lain yang damai.",
        "theological_basis": "Nama Allah 'As-Salam' (Sumber Kedamaian) dan perintah untuk merespons ajakan damai: 'وَإِن جَنَحُوا لِلسَّلَامِ فَاجْنَحْ لَهَا' (QS. Al-Anfal 8:61)."
    },
    "الْإِحْسَان": {
        "concept": "Berbuat baik (ihsan) melampaui keadilan; mencakup sikap santun, murah hati, dan empati terhadap semua orang, termasuk yang berbeda keyakinan.",
        "theological_basis": "Allah memerintahkan ihsan terhadap semua makhluk (QS. An-Nahl 16:90), dan Nabi Muhammad bersabda: 'أحسن الناس صحبة خيرهم للناس' (Orang terbaik adalah yang paling baik kepada manusia)."
    },
    "الْحُرِّيَّة": {
        "concept": "Kebebasan beragama adalah hak asasi manusia dalam Islam. Iman harus lahir dari kesadaran, bukan paksaan.",
        "theological_basis": "Prinsip 'لا إكراه في الدين' (QS. Al-Baqarah 2:256) menegaskan bahwa keyakinan tidak bisa dipaksakan. Jiwa manusia harus bebas memilih jalan hidupnya."
    },
    "الْحِكْمَة": {
        "concept": "Hikmah adalah pendekatan bijaksana dalam berdakwah dan berdialog antaragama, dengan mempertimbangkan konteks, budaya, dan kondisi lawan bicara.",
        "theological_basis": "Perintah berdakwah 'بالحكمة والموعظة الحسنة' (QS. An-Nahl 16:125) menunjukkan bahwa kebenaran harus disampaikan dengan cara yang paling mulia dan efektif."
    },
    "الْمُوَاطَنَة": {
        "concept": "Kehidupan bersama sebagai warga negara yang setara, saling menghormati, dan bekerja sama untuk kebaikan bersama, tanpa memandang agama.",
        "theological_basis": "QS. Al-Mumtahanah 60:8 menyatakan bahwa Allah tidak melarang berbuat baik dan adil terhadap mereka yang tidak memerangi kita: 'إِنَّ اللَّهَ يُحِبُّ الْمُقْسِطِينَ'."
    },
    "الْمُجَادَلَة بِالَّتِي هِيَ أَحْسَنُ": {
        "concept": "Dialog antaragama harus dilakukan dengan cara terbaik — penuh hormat, logis, dan tanpa ejekan — untuk membangun saling pengertian.",
        "theological_basis": "QS. Al-Ankabut 29:46: 'وَلَا تُجَادِلُوا أَهْلَ الْكِتَابِ إِلَّا بِالَّتِي هِيَ أَحْسَنُ' menekankan etika dialog dengan Ahli Kitab (Yahudi, Nasrani)."
    }
}

maqashid_map = {
    "Hifz ad-Din": ["دين","شرك","عباده","ايمان","توحيد","صلاه","زكاه","حج","القرآن","سنه","الله","اليوم الاخر","الجنه","النار","الوحي","النبوه","الرسول","المحبه في الله","الولاء والبراء"],
    "Hifz an-Nafs": ["قتل","حياه","نفس","صحه","سلامه","امن","حمايه","علاج","دواء","مأوى","القتال دفاعاً عن النفس","الاستشهاد","الاسعاف","الرعايه الصحيه","التطعيم","الوقايه","النظافه","الماء النقي","الهواء النقي"],
    "Hifz al-Aql": ["عقل","علم","فكر","تعليم","تعلم","بحث","دراسه","فهم","حكمه","تحليل","منطق","القراءه","الكتابه","الاستماع","الانصات","التحليل النقدي","الابتكار","الجامعه","المدرسه","اللغه","النحو","البلاغه","الفلسفه","العلوم العقليه"],
    "Hifz an-Nasl": ["نسل","عائله","زواج","ابناء","اطفال","تربيه","امومه","ابوه","الرضاعه","الحمل","الولاده","الطلاق","الوصايه","الاشراف","الزواج الاسلامي","الزواج المبكر","الاهمال","الايذاء","الحمايه من الاستغunakan"],
    "Hifz al-Mal": ["مال","رزق","استثمار","تجاره","عمل","وظيفه","زكاه","صدقه","وقف","الافلاس","الدين","التسامح في القرض","الغش","السرقه","الرشوه","الاحتيال","الربا","الزكاه الماليه","الانفاق في سبيل الله"]
}

toleransi_phrases = {
    "toleran": [
        "السلام", "الامان", "الامن", "الوئام", "الوفاق", "المحبة", "الود", "الرحمة", "الرضا", "الصفح", "العفو", "التسامح", "المسامحة", "الصفح", "الاحسان", "البر", "اللطف", "الرفق", "اللين", "الحُلم", "الصبر", "السكينة", "الطمأنينة", "الهدوء", "الاستقرار", "العدل", "القسط", "الإنصاف", "التوازن", "الاعتدال", "الوسطية", "التفاهم", "التعاون", "التآخي", "الاخاء", "المواساة", "التعاطف", "التعاضد", "التراحم", "التكافل", "الاحترام", "التكريم", "التقدير", "الإنصات", "الاستماع", "الحوار", "الكلام الطيب", "القول المعروف", "القول الحسن", "القول البليغ", "الحجة البيّنة", "الحجة الحسنة", "الدعوة بالحكمة", "بالتي هي أحسن", "لا إكراه في الدين", "لكم دينكم ولي دين", "لا تسبوا أهل الأديان", "من كان يؤمن بالله واليوم الآخر", "قل يا أهل الكتاب تعالوا إلى كلمة سواء", "وإذا خاطبهم الجاهلون قالوا سلاما", "ادخلوا في السلم كافة", "وألف بين قلوبهم", "كنتم خير أمة", "لكل أمة جعلنا منسكا", "من يصلي ويصوم", "ولا تجادلوا أهل الكتاب إلا بالتي هي أحسن", "الذين لا يشهدون الزور", "وهم على صلاتهم يحافظون", "ويؤثرون على أنفسهم ولو كان بهم خصاصة", "ويكظمون الغيظ", "والعافين عن الناس", "والله يحب المحسنين", "وأصلح ذات بينهم", "وأوفوا بعهد الله", "وأوفوا بالعهد", "وأوفوا باليمين", "ولن ترضى عنك اليهود ولا النصارى حتى تتبع ملتهم", "قل إن هدى الله هو الهدى", "وتعاونوا على البر والتقوى", "واعتصموا بحبل الله جميعا", "ولا تنازعوا", "واذكروا نعمة الله عليكم"
    ],
    "netral": [
        "ذكر", "نقل", "قراءة", "تحليل", "تفسير", "تأويل", "تفكر", "تدبر", "تعقل", "تأمل", "دراسة", "بحث", "تحقيق", "علم", "معرفة", "فهم", "إدراك", "وعي", "إحصاء", "تسجيل", "وصف", "بيان", "سياق", "زمن", "مكان", "ظروف", "واقع", "ظاهرة", "حدث", "واقعة", "قصة", "خبر", "حديث", "رواية", "إسناد", "متن", "نص", "لفظ", "معنى", "دلالات", "مقاصد", "حكم", "علة", "سبب", "غاية", "هدف", "مصلحة", "ضابط", "قاعدة", "منهج", "نظرية", "مدرسة", "فرقة", "طائفة", "جماعة", "مجتمع", "ثقافة", "تقليد", "عرف", "عادة", "سيرة", "تاريخ", "وقائع", "تطور", "تغير", "تحول", "تحليل", "مقارنة"
    ],
    "intoleran": [
        "كفر", "شرك", "نفاق", "مروق", "خروج", "بغى", "عتد", "ظلم", "عدوان", "قتل", "قتال", "حرب", "هجوم", "غزو", "تدمير", "تخريب", "حرق", "نهب", "سلب", "اغتصاب", "استعباد", "استرقاق", "إرهاب", "ترويع", "تهديد", "تخويف", "إكراه", "إجبار", "إرغام", "تعذيب", "تشريد", "تهجير", "إقصاء", "استئصال", "تقتيل", "ذبح", "تحريم", "شتم", "سب", "لعن", "شتيمة", "تحقير", "استهزاء", "استهانة", "استخفاف", "تكفير", "تجريد من الإنسانية", "تحريم", "اتهام باطل", "افتراء", "بهتان", "غش", "خداع", "تدليس", "تزييف", "تزوير"
    ]
}

ayat_dict = {
    "لا اكراه": ["QS. Al-Baqarah:256 'لا إكراه في الدين قد تبين الرشد من الغي'"],
    "لكم دينكم ولي دين": ["QS. Al-Kafirun:6 'لكم دينكم ولي دين'"],
    "الناس": ["QS. Al-Hujurat:13 'يا أيها الناس إنا خلقناكم من ذكر وأنثى...'"],
    "لا ينهاكم": ["QS. Al-Mumtahanah:8 'لا ينهاكم الله عن الذين لم يقاتلوكم...'"],
    "تجادلوا": ["QS. Al-Ankabut:46 'ولا تجادلوا أهل الكتاب إلا بالتي هي أحسن'"],
    "ما أنت عليهم بمضطر": ["QS. Al-Ghashiyah:22 'ما أنت عليهم بمضطر'"],
    "ولو شاء ربك": ["QS. Yunus:99 'ولو شاء ربك لآمن من في الأرض كلهم جميعاً'"],
    "ولا تسبوا": ["QS. Al-An'am:108 'ولا تسبوا الذين يدعون من دون الله'"]
}

# 10) DATASET TAFSIR (dipertahankan)
tafsir_data = [
    {
        "source": "al-Qurthubi,Al-Jami’ li Ahkam al-Qur’an, Juz 2, hlm. 216",
        "surah": "البقرة",
        "ayah": "256",
        "reference": "2:256",
        "theme": "لا إكراه في الدين",
        "text": "{لا إكراه في الدين}، هذه الآية من أوكد الأدلة على أن الدعوة إلى الله تكون بالحجة والبينة، لا بالإكراه. وقد نزلت في رجل من الأنصار أسلم وابناه كافران، فأراد أن يكرههما فنزلت هذه الآية تمنع من ذلك."
    },
    {
        "source": "az-Zuhaili, Al-Tafsir al-Munir, Juz 1, hlm. 478",
        "surah": "البقرة",
        "ayah": "256",
        "reference": "2:256",
        "theme": "لا إكراه في الدين",
        "text": "{لا إكراه في الدين}، هذه الآية الكريمة قول فصل في بطلان الإكراه في الدين، وأن الإيمان لا يكون إلا بطوع وشهادة قلب، لا بقسر أو جبر."
    },
    {
        "source": "al-Qurthubi, Al-Jami’ li Ahkam al-Qur’an, Juz 19, hlm. 302",
        "surah": "الكافرون",
        "ayah": "6",
        "reference": "109:6",
        "theme": "لكم دينكم ولي دين",
        "text": "{لكم دينكم ولي دين}، نزلت هذه السورة في رد النبي ﷺ على قومه الذين قالوا: تعال نعبد ما تعبد وتعبد ما نعبد. فأخبرهم الله أن الدين لا يجمع بين التناقض، وأمره بالبراءة من عبادتهم، ولكن باللين والإحسان."
    },
    {
        "source": "az-Zuhaili, Al-Tafsir al-Munir, Juz 13, hlm. 186",
        "surah": "الكافرون",
        "ayah": "6",
        "reference": "109:6",
        "theme": "لكم دينكم ولي دين",
        "text": "{لكم دينكم ولي دين}، قول النبي ﷺ هذا دليل صريح على أن كل طائفة على دينها، ولا يجب على النبي إكراههم، وإنما عليه التبليغ."
    },
    {
        "source": "al-Qurthubi, Al-Jami’ li Ahkam al-Qur’an, Juz 17, hlm. 231",
        "surah": "الحجرات",
        "ayah": "13",
        "reference": "49:13",
        "theme": "يا أيها الناس إنا خلقناكم",
        "text": "{يا أيها الناس إنا خلقناكم من ذكر وأنثى وجعلناكم شعوباً وقبائل لتعارفوا إن أكرمكم عند الله أتقاكم}، هذه الآية تنهى عن التفاخر بالأنساب والأعراق، وتبين أن التفاضل إنما هو بالتقوى."
    },
    {
        "source": "az-Zuhaili, Al-Tafsir al-Munir, Juz 12, hlm. 294",
        "surah": "الحجرات",
        "ayah": "13",
        "reference": "49:13",
        "theme": "يا أيها الناس إنا خلقناكم",
        "text": "{يا أيها الناس إنا خلقناكم}، الآية تبين أن الإنسان لا يتفضل على غيره بالعرق أو اللون، وإنما بالتقوى."
    },
    {
        "source": "al-Qurthubi, Al-Jami’ li Ahkam al-Qur’an, Juz 18, hlm. 124",
        "surah": "الممتحنة",
        "ayah": "8",
        "reference": "60:8",
        "theme": "لا ينهاكم الله عن الذين لم يقاتلوكم",
        "text": "{لا ينهاكم الله عن الذين لم يقاتلوكم في الدين ولم يخرجوكم من دياركم أن تبروهم وتقسطوا إليهم}، أذن الله للمسلمين في بِرّ الكافرين وإحسانهم إليهم إذا لم يحاربوهم في الدين."
    },
    {
        "source": "az-Zuhaili, Al-Tafsir al-Munir, Juz 14, hlm. 112",
        "surah": "الممتحنة",
        "ayah": "8",
        "reference": "60:8",
        "theme": "لا ينهاكم الله عن الذين لم يقاتلوكم",
        "text": "{لا ينهاكم الله عن الذين لم يقاتلوكم في الدين ولم يخرجوكم من دياركم}، الآية تبين أن المسلم لا يُنهى عن بِرّ من لم يعادِه، ولو كان كافراً."
    },
    {
        "source": "al-Qurthubi, Al-Jami’ li Ahkam al-Qur’an, Juz 15, hlm. 188",
        "surah": "العنكبوت",
        "ayah": "46",
        "reference": "29:46",
        "theme": "ولا تجادلوا أهل الكتاب إلا بالتي هي أحسن",
        "text": "{ولا تجادلوا أهل الكتاب إلا بالتي هي أحسن}، أمر الله نبيه ألا يجادرل أهل الكتاب إلا بالحجة البالغة والخطاب الحسن."
    },
    {
        "source": "az-Zuhaili, Al-Tafsir al-Munir, Juz 11, hlm. 243",
        "surah": "العنكبوت",
        "ayah": "46",
        "reference": "29:46",
        "theme": "ولا تجادلوا أهل الكتاب إلا بالتي هي أحسن",
        "text": "{ولا تجادلوا أهل الكتاب إلا بالتي هي أحسن}، الآية تدعو إلى الحوار المتَمَدِّن مع أهل الكتاب."
    },
    {
        "source": "al-Qurthubi, Al-Jami’ li Ahkam al-Qur’an, Juz 10, hlm. 312",
        "surah": "يونس",
        "ayah": "41",
        "reference": "10:41",
        "theme": "ولو شاء ربك لآمن من في الأرض كلهم جميعاً",
        "text": "{ولو شاء ربك لآمن من في الأرض كلهم جميعاً. أفأنت تُكره الناس حتى يكونوا مؤمنين؟}، آية واضحة على bahwa الإيمان لا يكون بالإكراه."
    },
    {
        "source": "az-Zuhaili, Al-Tafsir al-Munir, Juz 4, hlm. 305",
        "surah": "يونس",
        "ayah": "41",
        "reference": "10:41",
        "theme": "ولو شاء ربك لآمن من في الأرض كلهم جميعاً",
        "text": "{ولو شاء ربك لآمن من في الأرض كلهم جميعاً. أفأنت تُكره الناس حتى يكونوا مؤمنين؟}، هذه الآية تنفي الإكراه في الدين."
    },
    {
        "source": "al-Qurthubi, Al-Jami’ li Ahkam al-Qur’an, Juz 7, hlm. 155",
        "surah": "الأنعام",
        "ayah": "108",
        "reference": "6:108",
        "theme": "ولا تسبوا الذين يدعون من دون الله",
        "text": "{ولا تسبوا الذين يدعون من دون الله فيسبوا الله عدواً بغير علم}، نهى الله تعالى عن سب آلهة المشركين، خشية أن يقابلوا ذلك بسب الله تعالى."
    },
    {
        "source": "az-Zuhaili, Al-Tafsir al-Munir, Juz 3, hlm. 287",
        "surah": "الأنعام",
        "ayah": "108",
        "reference": "6:108",
        "theme": "ولا تسبوا الذين يدعون من دون الله",
        "text": "{ولا تسبوا الذين يدعون من دون الله}، هذه الآية تنهى عن سب الآلهة yang يعبدها غير المسلمين."
    }
]

all_texts = [item["text"] for item in tafsir_data]

# 4) NORMALISASI TEKS ARAB (dasar)
def normalize_arabic(text):
    if not text or not isinstance(text, str):
        return ""
    text = text.replace('\u00A0', ' ')
    text = re.sub(r'\u200f|\u200e', '', text)  # directional marks
    text = re.sub(r'ـ', '', text)  # tatweel
    text = re.sub(r'[إأٱآا]', 'ا', text)
    text = re.sub(r'ؤ', 'ء', text)
    text = re.sub(r'ئ', 'ي', text)
    text = re.sub(r'ى', 'ي', text)
    text = re.sub(r'ة', 'ه', text)
    text = re.sub(r'ٰ', '', text)  # sup-alef
    text = re.sub(r'[ًٌٍَُِّۚۖۗۘۙۜ]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# 4b) NORMALISASI UNTUK MATCHING (lebih robust)
def normalize_for_match(s):
    """Normalisasi yang lebih robust untuk pencocokan substring."""
    if not s or not isinstance(s, str):
        return ""
    # replace non-breaking & directional marks
    s = s.replace('\u00A0', ' ')
    s = re.sub(r'\u200f|\u200e', '', s)
    s = re.sub(r'ـ', '', s)  # tatweel
    # normalisasi huruf
    s = re.sub(r'[إأٱآا]', 'ا', s)
    s = re.sub(r'ؤ', 'ء', s)
    s = re.sub(r'ئ', 'ي', s)
    s = re.sub(r'ى', 'ي', s)
    s = re.sub(r'ة', 'ه', s)
    s = re.sub(r'ٰ', '', s)
    # hapus tanda baca & karakter non-arab (biarkan spasi)
    s = re.sub(r'[^\u0600-\u06FF\s]', ' ', s)
    # hapus harakat / tanda diakritik jika ada
    s = re.sub(r'[ًٌٍَُِّْـۚۖۗۘۙۜ]', '', s)
    s = re.sub(r'\s+', ' ', s)
    return s.strip().lower()

# 5) MATCH SENSITIF DENGAN FALLBACK SBERT
def match_sensitive_phrase_and_extract(text: str, sem_thresh=0.65) -> tuple:
    """
    Return: (override_probs, list_of_detected_phrases)
    - pertama coba exact / normalized substring match
    - jika tidak ada dan SBERT tersedia, coba similarity terhadap keys (fallback semantik)
    """
    if not text or len(str(text).strip()) < 3:
        return None, []
    clean_text = normalize_arabic(str(text))
    normalized = normalize_for_match(clean_text)
    detected_phrases = []
    # 1) substring matching (normalisasi pada keys juga)
    for phrase in SENSITIVE_VERSES.keys():
        phrase_norm = normalize_for_match(phrase)
        if phrase_norm and phrase_norm in normalized:
            detected_phrases.append(phrase)
    if detected_phrases:
        first_phrase = detected_phrases[0]
        return SENSITIVE_VERSES[first_phrase]["override_sentiment"], detected_phrases
    # 2) fallback semantik menggunakan SBERT (jika tersedia)
    try:
        if sbert_loaded:
            keys = list(SENSITIVE_VERSES.keys())
            emb_input = sbert_model.encode([normalized])[0]
            emb_keys = sbert_model.encode([normalize_for_match(k) for k in keys])
            sims = cosine_similarity([emb_input], emb_keys)[0]
            best_idx = int(np.argmax(sims))
            best_sim = float(sims[best_idx])
            if best_sim >= sem_thresh:
                matched = keys[best_idx]
                return SENSITIVE_VERSES[matched]["override_sentiment"], [matched + f" (sem_sim={best_sim:.3f})"]
    except Exception as e:
        print(f"⚠️ SBERT fallback error: {e}")
    return None, []

# 6) LOAD MODEL
print("📥 Memuat AraBERT...")
try:
    arabert_tok = AutoTokenizer.from_pretrained(MODEL_ARABERT)
    arabert_mod = AutoModelForSequenceClassification.from_pretrained(MODEL_ARABERT)
    pipe_arabert = pipeline("sentiment-analysis", model=arabert_mod, tokenizer=arabert_tok, device=device)
    arabert_loaded = True
    print("✅ AraBERT siap.")
except Exception as e:
    arabert_err = str(e)
    print(f"⚠️ Gagal memuat AraBERT: {arabert_err}")
print("📥 Memuat CAMeLBERT...")
try:
    camel_tok = AutoTokenizer.from_pretrained(MODEL_CAMEL)
    camel_mod = AutoModelForSequenceClassification.from_pretrained(MODEL_CAMEL)
    pipe_camel = pipeline("sentiment-analysis", model=camel_mod, tokenizer=camel_tok, device=device)
    camel_loaded = True
    print("✅ CAMeLBERT siap.")
except Exception as e:
    camel_err = str(e)
    print(f"⚠️ Gagal memuat CAMeLBERT: {camel_err}")
print("📥 Memuat SBERT...")
try:
    sbert_model = SentenceTransformer(MODEL_SBERT)
    sbert_loaded = True
    print("✅ SBERT siap (digunakan untuk kemiripan semantik & fallback ayat).")
except Exception as e:
    sbert_err = str(e)
    print(f"⚠️ Gagal memuat SBERT: {sbert_err}")

# 7) UTILITAS VISUALISASI
def make_bar_image(series, title):
    fig, ax = plt.subplots(figsize=(6, 4))
    try:
        if isinstance(series, pd.Series) and (series.empty or series.sum() == 0):
            ax.text(0.5, 0.5, 'No Data', ha='center', va='center')
            ax.axis('off')
        else:
            if isinstance(series, pd.Series):
                series.plot(kind='bar', ax=ax)
            else:
                pd.Series(series).plot(kind='bar', ax=ax)
            ax.set_title(title)
            ax.set_ylabel('Jumlah')
            plt.xticks(rotation=0)
        plt.tight_layout()
        buf = BytesIO()
        fig.savefig(buf, format='png', bbox_inches='tight', dpi=100)
        plt.close(fig)
        buf.seek(0)
        return Image.open(buf)
    except Exception as e:
        plt.close(fig)
        buf = BytesIO()
        img = Image.new('RGB', (400,200), color=(255,255,255))
        return img

def make_confusion_matrix_image(y_true, y_pred, labels, title="Confusion Matrix"):
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    fig, ax = plt.subplots(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels, ax=ax)
    ax.set_title(title)
    ax.set_xlabel('Prediksi')
    ax.set_ylabel('Aktual')
    plt.tight_layout()
    buf = BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight', dpi=100)
    plt.close(fig)
    buf.seek(0)
    return Image.open(buf)

# 8) RULE-BASED & MAPPING
def rule_based_probs(text_clean: str):
    pos_hits = sum(text_clean.count(p) for p in toleransi_phrases.get("toleran", []))
    neg_hits = sum(text_clean.count(p) for p in toleransi_phrases.get("intoleran", []))
    neu_hits = sum(text_clean.count(p) for p in toleransi_phrases.get("netral", []))
    total = pos_hits + neg_hits + neu_hits
    if total == 0:
        return {"positive": 0.2, "neutral": 0.6, "negative": 0.2}
    return {
        "positive": pos_hits / total,
        "neutral": neu_hits / total,
        "negative": neg_hits / total
    }

def softmax_scores_from_pipeline(pipe, text):
    try:
        out = pipe(text, truncation=True, max_length=512, return_all_scores=True)
        return out[0] if isinstance(out, list) else out
    except Exception:
        return [{"label": "neutral", "score": 0.5}]

def map_arabert_to_three(vec):
    three = {"positive": 0.0, "negative": 0.0, "neutral": 0.0}
    for item in vec:
        lab = item.get("label", "").strip().lower()
        sc = float(item.get("score", 0.0) or 0.0)
        if "pos" in lab:
            three["positive"] += sc
        elif "neg" in lab:
            three["negative"] += sc
        else:
            three["neutral"] += sc
    return three

def map_camel_to_three(vec):
    return map_arabert_to_three(vec)

def probs_to_label(probs: dict):
    if all(v == 0 for v in probs.values()):
        return "Netral", 0.0
    lab = max(probs, key=probs.get)
    mapping = {"positive": "Toleran", "neutral": "Netral", "negative": "Intoleran"}
    return mapping.get(lab, "Netral"), probs[lab]

# 9) PREDIKSI SENTIMEN UTAMA
def predict_sentiment(clean_text: str, mode: str):
    override_probs, matched_info = match_sensitive_phrase_and_extract(clean_text)
    if override_probs is not None:
        return override_probs, f"Override: {matched_info[0]}"
    if not arabert_loaded and not camel_loaded:
        return rule_based_probs(clean_text), "Rule-based Fallback"
    if mode == "AraBERT only" and arabert_loaded:
        v_ar = softmax_scores_from_pipeline(pipe_arabert, clean_text)
        return map_arabert_to_three(v_ar), "AraBERT"
    elif mode == "CAMeLBERT only" and camel_loaded:
        v_cm = softmax_scores_from_pipeline(pipe_camel, clean_text)
        return map_camel_to_three(v_cm), "CAMeLBERT"
    else:
        probs = {"positive": 0.0, "neutral": 0.0, "negative": 0.0}
        count = 0
        if arabert_loaded:
            v_ar = softmax_scores_from_pipeline(pipe_arabert, clean_text)
            ar_mapped = map_arabert_to_three(v_ar)
            for k in probs: probs[k] += ar_mapped[k]; count += 1
        if camel_loaded:
            v_cm = softmax_scores_from_pipeline(pipe_camel, clean_text)
            cm_mapped = map_camel_to_three(v_cm)
            for k in probs: probs[k] += cm_mapped[k]; count += 1
        if count == 0:
            rb = rule_based_probs(clean_text)
            for k in probs: probs[k] = rb[k]
        else:
            for k in probs: probs[k] /= count
        return probs, "Ensemble"

# 11) ANALISIS HERMENEUTIK ENHANCED
def analyze_hermeneutics_enhanced(text: str):
    clean_text = normalize_arabic(text)
    results = {"intertextual": [], "historical": [], "philosophical": [], "semantic_similarity": [], "topic_modeling": []}
    INTERTEXTUAL_DB_small = {
        "لا إكراه في الدين": {
            "related_verses": ["QS. Yunus:99", "QS. Al-Kahf:29"],
            "theme": "Kebebasan Iman",
            "explanation": "Dasar teologis bahwa iman harus lahir dari kesadaran, bukan paksaan."
        },
        "لكم دينكم ولي دين": {
            "related_verses": ["QS. Al-Kafirun:6"],
            "theme": "Pluralisme Agama",
            "explanation": "Prinsip penghormatan terhadap keyakinan lain."
        }
    }
    HISTORICAL_CONTEXT_DB_small = {
        "لا إكراه في الدين": {
            "context": "Diturunkan saat seorang sahabat ingin memaksa anaknya masuk Islam.",
            "significance": "Menegaskan larangan pemaksaan agama dalam praktik sosial."
        }
    }
    PHILOSOPHICAL_THEOLOGICAL_DB = {
        "لا إكراه في الدين": {
            "concept": "Kebebasan Iman",
            "explanation": "Iman yang dipaksakan bertentangan dengan sifat takwa yang harus lahir dari kesadaran pribadi."
        },
        "يا أيها الناس إنا خلقناكم": {
            "concept": "Kesetaraan Manusia",
            "explanation": "Nilai manusia ditentukan oleh ketakwaan, bukan ras, suku, atau garis keturunan."
        },
        "التسامح": {
            "concept": "Toleransi dalam Islam bukan sekadar membiarkan, tetapi aktif menghargai perbedaan, menjaga martabat, dan membangun kedamaian antarumat beragama.",
            "significance": "Konsep inklusivitas dan keamanan kolektif."
        }
    }
    # intertextual
    for keyword, data in INTERTEXTUAL_DB_small.items():
        if normalize_for_match(keyword) in normalize_for_match(clean_text):
            results["intertextual"].append({"keyword": keyword, "related_verses": data["related_verses"], "theme": data["theme"], "explanation": data["explanation"]})
    # historical
    for term, context in HISTORICAL_CONTEXT_DB_small.items():
        if normalize_for_match(term) in normalize_for_match(clean_text):
            results["historical"].append({"term": term, "context": context["context"], "significance": context["significance"]})
    # philosophical
    for concept, info in PHILOSOPHICAL_THEOLOGICAL_DB.items():
        if normalize_for_match(concept) in normalize_for_match(clean_text):
            results["philosophical"].append({"concept": info["concept"], "explanation": info["explanation"]})
    # semantic similarity SBERT
    try:
        if sbert_loaded:
            current_emb = sbert_model.encode([clean_text])
            all_embs = [sbert_model.encode([normalize_arabic(t)])[0] for t in all_texts]
            all_embs = np.vstack(all_embs)
            sims = cosine_similarity(current_emb, all_embs)[0]
            top_indices = np.argsort(sims)[-3:][::-1]
            for idx in top_indices:
                results["semantic_similarity"].append({"similarity": float(sims[idx]), "text": all_texts[idx][:100] + "..."})
    except Exception as e:
        results["semantic_similarity"].append({"error": f"SBERT error: {str(e)}"})
    # topic modeling LDA
    try:
        words = [w for w in re.findall(r'[\u0600-\u06FF]+', clean_text) if len(w) > 2]
        if len(words) > 5:
            dictionary = corpora.Dictionary([words])
            if len(dictionary) < 2:
                results["topic_modeling"].append({"warning": "Kosakata terlalu sedikit untuk LDA."})
            else:
                corpus = [dictionary.doc2bow(words)]
                lda_model = LdaModel(corpus, num_topics=1, id2word=dictionary, passes=10, random_state=42)
                topics = lda_model.print_topics(num_words=5)
                for topic_id, keywords in topics:
                    results["topic_modeling"].append({"topic_id": topic_id, "keywords": keywords})
        else:
            results["topic_modeling"].append({"warning": "Teks terlalu pendek untuk LDA."})
    except Exception as e:
        results["topic_modeling"].append({"error": f"LDA error: {str(e)}"})
    return results

def format_hermeneutics_results_enhanced(hermeneutics):
    output = ""
    if hermeneutics.get("intertextual"):
        output += "📖 **ANALISIS INTERTEKSTUAL:**\n"
        for item in hermeneutics["intertextual"]:
            output += f"• **{item['keyword']}**: {item['explanation']}\n"
    if hermeneutics.get("historical"):
        output += "🏺 **KONTEKS HISTORIS:**\n"
        for item in hermeneutics["historical"]:
            output += f"• **{item['term']}**: {item['context']}\n"
    if hermeneutics.get("philosophical"):
        output += "🧠 **ANALISIS FILOSOFIS-TEOLOGIS:**\n"
        for item in hermeneutics["philosophical"]:
            output += f"• **{item['concept']}**: {item['explanation']}\n"
    if hermeneutics.get("semantic_similarity"):
        output += "🔗 **KEMIRIPAN SEMANTIK (SBERT):**\n"
        for item in hermeneutics["semantic_similarity"]:
            if "error" in item:
                output += f"⚠️ {item['error']}\n"
            else:
                output += f"• Sim={item['similarity']:.3f}: {item['text']}\n"
    if hermeneutics.get("topic_modeling"):
        output += "📊 **TOPIC MODELING (LDA):**\n"
        for item in hermeneutics["topic_modeling"]:
            if "error" in item or "warning" in item:
                output += f"⚠️ {item.get('error', item.get('warning'))}\n"
            else:
                output += f"• Topic {item['topic_id']}: {item['keywords']}\n"
    return output if output else "Tidak ditemukan analisis hermeneutik yang signifikan."

# 12) ANALISIS UTAMA (patched)
def analyze_text(input_text: str, mode: str):
    if not input_text or len(str(input_text).strip()) < 3:
        df_empty = pd.DataFrame({"Kata": [], "Kategori": []})
        img_empty = make_bar_image(pd.Series(dtype=int), "Tidak ada data")
        return "Input tidak valid.", df_empty, "Input kosong.", "-", "-", pd.DataFrame(), "⚠️ Tidak ada", img_empty, img_empty, pd.DataFrame(), "Tidak ada analisis"
    try:
        clean_text = normalize_arabic(str(input_text))
        # Deteksi ayat sensitif
        override_probs, ayat_detected = match_sensitive_phrase_and_extract(clean_text)
        if override_probs is not None:
            probs = override_probs
            used = f"Override: {ayat_detected[0]}"
        else:
            probs, used = predict_sentiment(clean_text, mode)
        label, score = probs_to_label(probs)
        highlighted = clean_text
        for w in sorted(toleransi_phrases.get("toleran", []), key=len, reverse=True):
            if w in highlighted:
                highlighted = highlighted.replace(w, f" [TOL:{w}] ")
        for w in sorted(toleransi_phrases.get("intoleran", []), key=len, reverse=True):
            if w in highlighted:
                highlighted = highlighted.replace(w, f" [INT:{w}] ")
        words = re.findall(r'[\u0600-\u06FF]+', clean_text)
        word_count = len(words)
        sents = [s.strip() for s in re.split(r'[.!?;،]', clean_text) if len(s.strip()) > 5]
        sentence_count = len(sents)
        data = []
        for w in words:
            cat = "Toleran" if any(p in w for p in toleransi_phrases["toleran"]) \
                else "Intoleran" if any(p in w for p in toleransi_phrases["intoleran"]) \
                else "Netral" if any(p in w for p in toleransi_phrases["netral"]) else "Lainnya"
            data.append({"Kata": w, "Kategori": cat})
        df = pd.DataFrame(data)
        counts = df["Kategori"].value_counts().reindex(["Toleran", "Netral", "Intoleran", "Lainnya"], fill_value=0)
        img_dist = make_bar_image(counts, "Distribusi Kategori")
        sent_res = []
        for s in sents:
            p, _ = predict_sentiment(normalize_arabic(s), mode)
            lab, cf = probs_to_label(p)
            sent_res.append({"Kalimat": s, "Label": lab, "P(+)": round(p["positive"],3)})
        df_sent = pd.DataFrame(sent_res) if sent_res else pd.DataFrame(columns=["Kalimat","Label","P(+)"])
        if not df_sent.empty:
            counts_sent = df_sent["Label"].value_counts().reindex(["Toleran", "Netral", "Intoleran"], fill_value=0)
            img_sent = make_bar_image(counts_sent, "Sentimen per Kalimat")
        else:
            img_sent = make_bar_image(pd.Series(dtype=int), "Tidak Ada Kalimat")
        kw_all = list(set(toleransi_phrases.get("toleran", []) + toleransi_phrases.get("intoleran", [])))
        kwic_rows = []
        for kw in kw_all:
            for m in re.finditer(re.escape(kw), clean_text):
                start = max(m.start()-25, 0)
                end = min(m.end()+25, len(clean_text))
                ctx = clean_text[start:end]
                kwic_rows.append({"Keyword": kw, "KWIC": ctx})
        df_kwic = pd.DataFrame(kwic_rows) if kwic_rows else pd.DataFrame(columns=["Keyword", "KWIC"])
        # --- PATCH: Ayat Penguat tabel
        if ayat_detected:
            rows = []
            for ph in ayat_detected:
                data = SENSITIVE_VERSES.get(ph, {})
                if "intertextual" in data:
                    for rv in data["intertextual"].get("related_verses", []):
                        rows.append({
                            "Frasa Utama": ph,
                            "Ayat Terkait": rv,
                            "Penjelasan": data["intertextual"].get("explanation", "-"),
                            "Konteks": data.get("context", "-")
                        })
                else:
                    rows.append({"Frasa Utama": ph, "Ayat Terkait": "-", "Penjelasan": "-", "Konteks": data.get("context","-")})
            df_ayat = pd.DataFrame(rows)
        else:
            df_ayat = pd.DataFrame(columns=["Frasa Utama", "Ayat Terkait", "Penjelasan", "Konteks"])
        # --- PATCH: Peringatan Kontekstual naratif
        if ayat_detected:
            warns = []
            for ph in ayat_detected:
                data = SENSITIVE_VERSES.get(ph, {})
                warns.append(f"📖 {ph}")
                if "historical_context" in data:
                    warns.append(f"   🟡 Historis: {data['historical_context']}")
                maq = data.get("maqashid", [])
                if maq:
                    warns.append(f"   🎯 Maqashid terkait: {', '.join(maq)}")
                warns.append("   🟢 Perhatikan konteks agar tidak disalahpahami.")
            warn_text = "\n".join(warns)
        else:
            warn_text = "⚠️ Tidak ada peringatan kontekstual."
        stats = (
            f"Jumlah Kata: {word_count}\n"
            f"Jumlah Kalimat: {sentence_count}\n"
            f"Model: {used}\n"
            f"Sentimen: {label} (confidence={score:.3f})\n"
            f"P(+)={probs['positive']:.3f}| P(-)={probs['negative']:.3f}| P(0)={probs['neutral']:.3f}"
        )
        ctx = ", ".join([p for p in toleransi_phrases.get("toleran", []) if p in clean_text]) or "-"
        maq = ", ".join([m for m, kws in maqashid_map.items() if any(kw in clean_text for kw in kws)]) or "-"
        ayat = ", ".join(ayat_detected) if ayat_detected else "-"
        warn = "⚠️ Tidak ada peringatan kontekstual." if not ayat else f"🟢 Override aktif: {ayat}"
        hermeneutic_analysis = format_hermeneutics_results_enhanced(analyze_hermeneutics_enhanced(clean_text))
        return highlighted, df, stats, ctx, maq, df_ayat, warn_text, img_dist, img_sent, df_kwic, hermeneutic_analysis
    except Exception as e:
        print(f"❌ ERROR di analyze_text: {str(e)}")
        df_err = pd.DataFrame({"Kata": [], "Kategori": []})
        img_err = make_bar_image(pd.Series([1], index=["Error"]), "Error")
        return f"Error: {str(e)}", df_err, "Error", "-", "-", pd.DataFrame(), "🔴 Exception", img_err, img_err, pd.DataFrame(), "Error runtime"

# 13) EVALUASI CSV
def evaluate_csv(file_path, mode):
    df = pd.read_csv(file_path.name) if hasattr(file_path, 'name') else pd.read_csv(file_path)
    if "text" not in df.columns or "gold" not in df.columns:
        raise ValueError("CSV harus memiliki kolom: 'text' dan 'gold'.")
    def normalize_gold(label):
        label = str(label).strip().lower()
        if "toleran" in label: return "Toleran"
        elif "intoleran" in label: return "Intoleran"
        else: return "Netral"
    y_true_raw = df["gold"].apply(normalize_gold)
    y_true = pd.Series(y_true_raw)
    preds, probs_list = [], []
    for txt in df["text"]:
        clean = normalize_arabic(txt)
        probs, _ = predict_sentiment(clean, mode)
        preds.append(probs_to_label(probs)[0])
        probs_list.append(probs)
    y_pred = pd.Series(preds)
    acc = accuracy_score(y_true, y_pred)
    pr, rc, f1, support = precision_recall_fscore_support(y_true, y_pred, labels=["Toleran", "Netral", "Intoleran"], zero_division=0)
    report = classification_report(y_true, y_pred, labels=["Toleran", "Netral", "Intoleran"], zero_division=0)
    cm_img = make_confusion_matrix_image(y_true, y_pred, labels=["Toleran", "Netral", "Intoleran"])
    out_df = df.copy()
    out_df["pred"] = y_pred
    out_df["P(+)"] = [round(p["positive"], 4) for p in probs_list]
    out_df["P(0)"] = [round(p["neutral"], 4) for p in probs_list]
    out_df["P(-)"] = [round(p["negative"], 4) for p in probs_list]
    out_path = "hasil_prediksi.csv"
    out_df.to_csv(out_path, index=False)
    summary = (
        f"Mode: {mode}\n"
        f"Accuracy: {acc:.4f}\n"
        f"Precision: {pr}\nRecall: {rc}\nF1: {f1}\nSupport: {support}\n"
        f"Classification Report:\n{report}"
    )
    return summary, cm_img, out_path

# 14) PERBANDINGAN TAFSIR
def compare_tafsir(theme_key="لا إكراه في الدين"):
    results = []
    for item in tafsir_data:
        if theme_key.lower() in item["theme"].lower() or theme_key == "all":
            clean_text = normalize_arabic(item["text"])
            probs, used = predict_sentiment(clean_text, "Ensemble")
            label, conf = probs_to_label(probs)
            ctx = ", ".join([p for p in toleransi_phrases.get("toleran", []) if p in clean_text]) or "-"
            maq = ", ".join([m for m, kws in maqashid_map.items() if any(kw in clean_text for kw in kws)]) or "-"
            results.append({
                "Sumber": item["source"],
                "Ayat": f"{item['surah']}:{item['ayah']}",
                "Tema": item["theme"],
                "Sentimen": label,
                "P(Toleran)": round(probs["positive"], 3),
                "Maqashid": maq,
                "Frasa Toleran": ctx
            })
    return pd.DataFrame(results)

# 15) GRADIO DASHBOARD
with gr.Blocks(theme=gr.themes.Soft(), title="Dashboard: Analisis Toleransi Tafsir") as demo:
    gr.Markdown("## 🕌 DASHBOARD: ANALISIS SENTIMEN TOLERANSI BERAGAMA DALAM TAFSIR AL-QUR'AN")
    with gr.Tab("🔍 Analisis Tunggal"):
        inp = gr.Textbox(label="Teks Tafsir (Arab)", lines=7, placeholder="اكتب النص هنا...")
        mode = gr.Dropdown(["Ensemble", "AraBERT only", "CAMeLBERT only"], value="Ensemble")
        btn = gr.Button("🔎 ANALISIS", variant="primary")
        out_text = gr.Textbox(label="🔤 Teks + Highlight", lines=6)
        out_table = gr.DataFrame(label="📊 Tabel Kata")
        out_stats = gr.Textbox(label="📈 Statistik", lines=6)
        out_ctx = gr.Textbox(label="📌 Frasa Toleran", lines=3)
        out_maq = gr.Textbox(label="🎯 Maqashid", lines=3)
        out_ayat = gr.DataFrame(label="📖 Ayat Penguat")   # ✅ sekarang DataFrame
        out_warn = gr.Textbox(label="⚠️ Peringatan Kontekstual", lines=8)
        out_plot = gr.Image(label="🖼️ Distribusi Kata")
        out_plot_sent = gr.Image(label="📉 Sentimen per Kalimat")
        out_kwic = gr.DataFrame(label="🔎 KWIC: Keyword-in-Context")
        out_herm = gr.Textbox(label="🔍 Hermeneutik (Lengkap)", lines=12)
        btn.click(fn=analyze_text, inputs=[inp, mode],
                  outputs=[out_text, out_table, out_stats, out_ctx, out_maq, out_ayat, out_warn, out_plot, out_plot_sent, out_kwic, out_herm])
    with gr.Tab("📊 Bandingkan Tafsir"):
        gr.Markdown("### 📊 Bandingkan Tafsir al-Qurthubi & az-Zuhaili — Ayat Toleransi (Data Asli)")
        theme_choice = gr.Dropdown(
            choices=[
                "لا إكراه في الدين",
                "لكم دينكم ولي دين",
                "يا أيها الناس إنا خلقناكم",
                "لا ينهاكم الله عن الذين لم يقاتلوكم",
                "ولا تجادلوا أهل الكتاب إلا بالتي هي أحسن",
                "ولو شاء ربك لآمن من في الأرض كلهم جميعاً",
                "ولا تسبوا الذين يدعون من دون الله",
                "all"
            ],
            value="لا إكراه في الدين",
            label="Pilih Tema Ayat"
        )
        btn_comp = gr.Button("📊 BANDINGKAN")
        out_comp_table = gr.DataFrame()
        btn_comp.click(compare_tafsir, theme_choice, out_comp_table)
    with gr.Tab("🧪 Evaluasi CSV"):
        file_csv = gr.File(label="Upload CSV (.csv)")
        mode_eval = gr.Dropdown(["Ensemble", "AraBERT only", "CAMeLBERT only"], value="Ensemble")
        btn_eval = gr.Button("📐 Evaluasi")
        out_summary = gr.Textbox(lines=18)
        out_cm = gr.Image()
        out_dl = gr.File()
        btn_eval.click(evaluate_csv, [file_csv, mode_eval], [out_summary, out_cm, out_dl])
    with gr.Tab("ℹ️ Tentang"):
        about = (
            "Pipeline: Hybrid Ensemble + Rule-based + Hermeneutik\n"
            f"Status: CAMeLBERT={'OK' if camel_loaded else 'FAIL'}, AraBERT={'OK' if arabert_loaded else 'FAIL'}, SBERT={'OK' if sbert_loaded else 'FAIL'}\n"
            f"Fitur: Analisis Tunggal, Perbandingan Tafsir, Evaluasi, Hermeneutik\n"
            f"Error CAMeLBERT: {camel_err or '-'}\n"
            f"Error AraBERT: {arabert_err or '-'}\n"
            f"Error SBERT: {sbert_err or '-'}\n"
            "📚 Sumber Tafsir:\n"
            "- al-Qurthubi: Al-Jami’ li-Ahkam al-Qur’an\n"
            "- az-Zuhaili: Al-Tafsir al-Munir"
        )
        gr.Textbox(value=about, label="Info", lines=14, interactive=False)

print("✅ Semua siap. Menjalankan dashboard...")
demo.launch(share=True, show_error=True)

📥 Memuat AraBERT...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/842 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ SBERT siap (digunakan untuk kemiripan semantik & fallback ayat).
✅ Semua siap. Menjalankan dashboard...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://84c234425626329688.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
